In [2]:
from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=int("2022"), day=int("19"))

puzzle.header()

# Not Enough Minerals

[Open Website](https://adventofcode.com/2022/day/19)

In [3]:

class Cost:
    def __init__(self, robot: str, costs: dict):
        self.robot = robot
        self.costs = costs

    def can_produce(self, state: dict) -> bool:
        for type in self.costs:
            if state[type] < self.costs[type]:
                return False
        return True

    def __repr__(self):
        return (f"Cost(robot={self.robot}, "
                f"costs={self.costs})")


# helper functions
class Blueprint:
    def __init__(self, costs: dict):
        self.costs = costs

    def __repr__(self):
        return (f"Blueprint(costs={self.costs})")


def domain_from_input(input: InputReader) -> set:
    lines = input.lines_as_str()
    # remove empty lines
    lines = [line for line in lines if line]
    blueprints = set()

    for i in range(0, len(lines), 5):
        costs = {}
        # parse ore robot using pattern
        line = lines[i + 1].strip(".").split()
        costs[line[1]] = {
            line[-1]: int(line[-2]),
        }

        # parse clay robot using pattern
        line = lines[i + 2].strip(".").split()
        costs[line[1]] = {
            line[-1]: int(line[-2]),
        }

        # parse obsidian robot using pattern
        line = lines[i + 3].strip(".").split()
        costs[line[1]] = {
            line[-1]: int(line[-2]),
            line[-4]: int(line[-5]),
        }

        # parse geode robot using pattern
        line = lines[i + 4].strip(".").split()
        costs[line[1]] = {
            line[-1]: int(line[-2]),
            line[-4]: int(line[-5]),
        }

        for type in costs.keys():
            costs[type] = Cost(type, costs[type])

        blueprint = Blueprint(costs)
        blueprints.add(blueprint)

    return blueprints


test_input = domain_from_input(puzzle.get_code_block(0))
print(test_input)

{Blueprint(costs={'ore': Cost(robot=ore, costs={'ore': 4}), 'clay': Cost(robot=clay, costs={'ore': 2}), 'obsidian': Cost(robot=obsidian, costs={'clay': 14, 'ore': 3}), 'geode': Cost(robot=geode, costs={'obsidian': 7, 'ore': 2})}), Blueprint(costs={'ore': Cost(robot=ore, costs={'ore': 2}), 'clay': Cost(robot=clay, costs={'ore': 3}), 'obsidian': Cost(robot=obsidian, costs={'clay': 8, 'ore': 3}), 'geode': Cost(robot=geode, costs={'obsidian': 12, 'ore': 3})})}


In [29]:
# test case (part 1)
def part_1(reader: InputReader, debug: bool) -> int:
    blueprints = domain_from_input(reader)
    total_geodes = 0
    max_steps = 24

    def score_robots(steps: int, robots: dict) -> int:
        step_weight = (max_steps - steps)
        return sum([robots[type] for type in robots]) + step_weight

    build_weights = {
        "ore": 1,
        "clay": 2,
        "obsidian": 5,
        "geode": 10
    }


    for blueprint in blueprints:
        if debug:
            print(blueprint)

        robots = {
            "ore": 1,
            "clay": 0,
            "obsidian": 0,
            "geode": 0,
        }
        supplies = {
            "ore": 0,
            "clay": 0,
            "obsidian": 0,
            "geode": 0
        }

        queue = []
        for next_robot in build_weights.keys():
            queue.append((next_robot, 0, score_robots(0, robots), robots.copy(), supplies.copy()))

        while queue:
            # sort queue
            queue = sorted(queue, key=lambda x: x[2], reverse=False)
            # get next
            next_robot, steps, score, robots, supplies = queue.pop()

            if debug and steps == 0:
                print(f"next_robot: {next_robot}, steps: {steps}, score: {score}, robots: {robots}, supplies: {supplies}")

            if steps == max_steps:
                if debug:
                    print(f"steps: {steps}, score: {score}, robots: {robots}, supplies: {supplies}")
                total_geodes += supplies["geode"]
                # clear queue
                queue = []
                continue

            # produce ore
            for type in robots:
                supplies[type] += robots[type]

            # product robots if possible
            if blueprint.costs[next_robot].can_produce(supplies):
                for type in blueprint.costs[next_robot].costs:
                    supplies[type] -= blueprint.costs[next_robot].costs[type]

                robots[next_robot] += 1
                score += 1
                for type in build_weights.keys():
                    queue.append((type, steps + 1, score_robots(steps, robots), robots.copy(), supplies.copy()))
            else:
                queue.append((next_robot, steps + 1, score_robots(steps, robots), robots.copy(), supplies.copy()))

    return total_geodes


result = part_1(puzzle.example(0), True)
print(result)
assert result == 33

Blueprint(costs={'ore': Cost(robot=ore, costs={'ore': 4}), 'clay': Cost(robot=clay, costs={'ore': 2}), 'obsidian': Cost(robot=obsidian, costs={'clay': 14, 'ore': 3}), 'geode': Cost(robot=geode, costs={'obsidian': 7, 'ore': 2})})
next_robot: geode, steps: 0, score: 25, robots: {'ore': 1, 'clay': 0, 'obsidian': 0, 'geode': 0}, supplies: {'ore': 0, 'clay': 0, 'obsidian': 0, 'geode': 0}
next_robot: obsidian, steps: 0, score: 25, robots: {'ore': 1, 'clay': 0, 'obsidian': 0, 'geode': 0}, supplies: {'ore': 0, 'clay': 0, 'obsidian': 0, 'geode': 0}
next_robot: clay, steps: 0, score: 25, robots: {'ore': 1, 'clay': 0, 'obsidian': 0, 'geode': 0}, supplies: {'ore': 0, 'clay': 0, 'obsidian': 0, 'geode': 0}
next_robot: ore, steps: 0, score: 25, robots: {'ore': 1, 'clay': 0, 'obsidian': 0, 'geode': 0}, supplies: {'ore': 0, 'clay': 0, 'obsidian': 0, 'geode': 0}
steps: 24, score: 23, robots: {'ore': 2, 'clay': 20, 'obsidian': 0, 'geode': 0}, supplies: {'ore': 0, 'clay': 190, 'obsidian': 0, 'geode': 0}
B

AssertionError: 

In [ ]:
# real case (part 1)
result = part_1(puzzle.input(), False)
print(result)

In [ ]:
# test case (part 2)
def part_2(reader: InputReader, debug: bool) -> int:
    lines = domain_from_input(reader)
    if debug:
        print(lines)
    return 0


result = part_2(puzzle.example(0), True)
print(result)
assert result == 0

In [ ]:
# real case (part 2)
result = part_2(puzzle.input(), False)
print(result)

In [ ]:
# print easters eggs
puzzle.print_easter_eggs()